In [ ]:
import os

In [ ]:
pathToDataset = os.path.join(os.getcwd(),"Snakes")
print(pathToDataset)

In [ ]:
trainPath = pathToDataset + "/train"
validPath = pathToDataset + "/valid"
print(trainPath)
print(validPath)

In [ ]:
noOfClasses = len(os.listdir(trainPath))
print(noOfClasses)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from keras.layers import GlobalAveragePooling2D,Dense,Dropout
from keras import regularizers
from keras.models import Model
from tensorflow.keras.optimizers import Adam



In [ ]:
train_dataGen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_dataGen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data = train_dataGen.flow_from_directory(directory=trainPath,
                                             batch_size = 32,
                                             target_size = (224,224),
                                             class_mode = "categorical")

valid_data = valid_dataGen.flow_from_directory(directory=validPath,
                                             batch_size = 32,
                                             target_size = (224,224),
                                             class_mode = "categorical")

In [ ]:
base_model = InceptionV3(weights='imagenet',include_top=False)

In [ ]:
print(base_model.input)
print(base_model.output)

In [ ]:
for layer in base_model.layers:
    print(layer.name,layer.trainable)

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x) # Added layer
x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x) # Added layer
predictions = Dense(10, activation='softmax')(x)

In [ ]:
print(predictions.shape)

In [ ]:
model=Model(base_model.input,predictions)

In [ ]:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [ ]:
for layer in model.layers:
    print(layer.name,layer.trainable)

In [ ]:
history = model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,epochs=10,validation_data=valid_data,)

In [ ]:
model.save("SnakeModel.h5")

In [ ]:
model.save("SnakeMode.keras")